In [ ]:
!pip install -U ogb
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-geometric
!pip install -U scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import torch_geometric
import urllib3
import outdated
import torch

!python -c "import ogb; print(ogb.__version__)"
!python --version
print(pd.__version__)
print(np.__version__)
print(torch.__version__)
print(torch.version.cuda)
print(torch_geometric.__version__)
print(urllib3.__version__)
print(outdated.__version__)

!python3 -m pip show scikit-learn

In [ ]:
import torch
import torch.nn.functional as F

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

In [ ]:
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

# GCN model with 2 layers 
class GCN(torch.nn.Module):

    # classes sorties, size des
    def __init__(self, data, num_classes_end, hidden=128, layers_gcn=4):

        super(GCN, self).__init__()

        self.num_layers_gcn = layers_gcn

        # liste des couches de convolution
        self.convs = torch.nn.ModuleList()
        # liste des batch size normalisation
        self.bns = torch.nn.ModuleList() 

        # ajout de la d'entrée 
        self.convs.append(GCNConv(data.num_node_features, hidden))   
        # meilleur taux d'apprentissage et évite overfitting (change un peu les données d'entrées à chaque epoch) 
        self.bns.append(torch.nn.BatchNorm1d(hidden))

        # profondeur du réseau (-2 car on enlève le début et la fin)
        # on empile les couches cachés
        for i in range(self.num_layers_gcn - 2):
            self.convs.append(GCNConv(hidden, hidden))
            self.bns.append(torch.nn.BatchNorm1d(hidden))

        # couche de sortie
        self.convs.append(GCNConv(hidden, num_classes_end))

    # reset les paramètres entre les runs
    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, data):
        # on récupère les features et la matrice d'adjacence
        x, adj_t = data.x, data.adj_t

        # on ajoute les couches de convolution
        for i in range(self.num_layers_gcn - 1):
            x = self.convs[i](x, adj_t)
            x = self.bns[i](x) 
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)

        # couche de sortie
        x = self.convs[-1](x, adj_t)

        # sort un vecteur de probabilités désiquilibré
        # log pour pas avoir des trop petites proba pour l'ordinateur
        # ex 0.00000000001 = 0 pour la machine
        x = F.log_softmax(x, dim=1)  
        return x

In [ ]:
def train(model, data, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    # on récupère les prédictions
    out = model(data)[train_idx]
    # squeeze pour mettre en forme le y sous forme de vecteur
    # nll loss permet de résoudre les porblème de classication avec c classe
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [ ]:
num_layers = 20
hidden_channels = 128
dropout = 0.5
lr = 0.01
epochs = 100
runs = 5

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

dataset = PygNodePropPredDataset(name='ogbn-arxiv', transform=T.ToSparseTensor())

In [ ]:
# splitting data
split_idx = dataset.get_idx_split()

# gets data
data = dataset[0]
data = data.to(device)

# permet d'enlever l'orientation du graphe
# car arxiv est grpahe orienté
data.adj_t = data.adj_t.to_symmetric()

# gets train features ids
train_idx = split_idx['train'].to(device)

# Create model
model = GCN(data, dataset.num_classes).to(device)

evaluator = Evaluator(name='ogbn-arxiv')

In [ ]:
log_steps = 1

best_acc_valid = []
best_acc_test = []

for i in range(runs):
  best_test = 0
  best_valid = 0
  model.reset_parameters()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  for epoch in range(1, 1 + epochs):
    loss = train(model, data, train_idx, optimizer)
    result = test(model, data, split_idx, evaluator)

    if epoch % log_steps == 0:
      train_acc, valid_acc, test_acc = result
      print(f'Run: {i + 1:02d}, '
            f'Epoch: {epoch:02d}, '
            f'Loss: {loss:.4f}, '
            f'Train: {100 * train_acc:.2f}%, '
            f'Valid: {100 * valid_acc:.2f}%, '
            f'Test: {100 * test_acc:.2f}%')
                    
    if test_acc > best_test:
      best_test = test_acc
    if valid_acc > best_valid:
      best_valid = valid_acc

  print(best_test, best_valid)
  best_acc_valid.append(best_valid)
  best_acc_test.append(best_test)


moy_valid = sum(best_acc_valid) / len(best_acc_valid)
moy_test = sum(best_acc_test) / len(best_acc_test)

print(f'Validation {moy_valid} Test {moy_test}')

In [ ]:
!nvidia-smi